# **Step 1: Data Scraping from B-Tech Group Website**

## a. dependancies

In [ ]:
!pip install --quiet google-colab-selenium --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 3.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
typeguard 4.4.4 requires typing_extensions>=4.14.0, but you have typing-extensions 4.13.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_mach

In [ ]:
API_KEY= "--"

In [ ]:
import time, requests, pandas as pd
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import google_colab_selenium as gs

## b. scrapping main functions

In [ ]:
def setup_driver():
    return gs.Chrome()

def expand_list(driver, clicks=3, wait=2):
    for i in range(clicks):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(wait)
        try:
            load_btn = driver.find_element(By.CSS_SELECTOR, "div.amscroll-load-button")
            if load_btn.is_displayed():
                driver.execute_script("arguments[0].click();", load_btn)
                print(f"→ loaded page {i+1}")
                time.sleep(wait)
            else:
                break
        except (NoSuchElementException, ElementClickInterceptedException):
            break

def extract_specs(driver):
    specs = {}
    try:
        block = driver.find_element(By.CSS_SELECTOR, "div.product.attribute.about-product")
        for detail in block.find_elements(By.CSS_SELECTOR, "div.detail"):
            key = detail.find_element(By.TAG_NAME, "strong").text.strip()
            vals = [p.text.strip() for p in detail.find_elements(By.TAG_NAME, "p")]
            specs[key] = " | ".join(vals)
    except NoSuchElementException:
        pass
    return specs

def scrape_btech_category(url, apikey, zenrows_endpoint="https://api.zenrows.com/v1/"):
    params = {"url": url, "apikey": apikey}
    resp = requests.get(zenrows_endpoint, params=params)
    print("ZenRows status:", resp.status_code)

    driver = setup_driver()
    driver.get(url)
    time.sleep(5)
    expand_list(driver, clicks=3, wait=2)

    product_links = []
    for item in driver.find_elements(By.CSS_SELECTOR, "div.product-item-view"):
        try:
            name  = item.find_element(By.CSS_SELECTOR, "h2.plpTitle").text.strip()
            price = item.find_element(By.CSS_SELECTOR, "span.special-price span.price-wrapper").text.strip()
            link  = item.find_element(By.CSS_SELECTOR, "a.listingWrapperSection").get_attribute("href")
            product_links.append({"name": name, "price": price, "link": link})
        except Exception:
            continue
    all_data = []
    for p in product_links:
        driver.get(p["link"])
        time.sleep(3)
        specs = extract_specs(driver)
        all_data.append({**p, **specs})

    driver.quit()

    df = pd.DataFrame(all_data)
    def combine_specs(row):
        return "\n".join(f"{k}: {v}" for k,v in row.items()
                         if k not in ("name","price","link") and pd.notnull(v))
    df["specs_combined"] = df.apply(combine_specs, axis=1)
    return df[["name","price","link","specs_combined"]]

**Links:-**

Dishwashers :: https://btech.com/ar/major-domestic-appliances/dishwashers.html

Phones :: https://btech.com/ar/moblies/mobile-phones-smartphones/smartphones.html

Fridges :: https://btech.com/ar/major-domestic-appliances/refrigerators/fridges.html

Deep_Freezers :: https://btech.com/ar/major-domestic-appliances/refrigerators/freezers.html

Cookers :: https://btech.com/ar/major-domestic-appliances/cookers.html

Tablets :: https://btech.com/ar/moblies/mobile-phones-smartphones/tablets.html

TVs :: https://btech.com/ar/tv-home-theater/screens.html

In [ ]:
CATEGORY_URL  = "https://btech.com/ar/tv-home-theater/screens.html"
df = scrape_btech_category(CATEGORY_URL, API_KEY)
df.to_csv("btech.csv", index=False, encoding="utf-8-sig")
df.head()

ZenRows status: 200


<IPython.core.display.Javascript object>

→ loaded page 1
→ loaded page 2
→ loaded page 3


,name,price,link,specs_combined
0,شاشة تلفزيون سامسونج سمارت 55 بوصة بدقة 4K UHD...,"19,099",https://btech.com/ar/samsung-55-4k-uhd-smart-l...,البراند: سامسونج\nحجم الشاشة: 55 بوصة\nنوع الت...
1,شاشة تلفزيون سامسونج سمارت 43 بوصة LED بدقة FH...,"13,500",https://btech.com/ar/samsung-43-fhd-smart-led-...,البراند: سامسونج\nنوع الدقة: FHD\nنوع الشاشة: ...
2,شاشة سامسونج 32 بوصة سمارت LED ، دقة HD، بريسي...,"8,323",https://btech.com/ar/samsung-32-hd-smart-led-u...,البراند: سامسونج\nنوع الدقة: HD (1366×768)\nال...
3,تلفزيون سمارت ال جي 55 بوصة، LED، دقة 4K UHD، ...,"22,799",https://btech.com/ar/lg-tv-55-inch-4k-smart-le...,البراند: ال جي\nحجم الشاشة: 55 بوصة\nنوع التلف...
4,تلفزيون ال جي، 70 بوصة، سمارت LED، دقة 4K UHD،...,"31,999",https://btech.com/ar/lg-70-inch-4k-smart-led-t...,البراند: ال جي\nنوع التلفزيون: سمارت\nنوع الشا...


## c. test url

In [ ]:
df.shape

(65, 8)

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving btech_fridges.csv to btech_fridges.csv


In [ ]:
df = pd.read_csv("btech_fridges.csv")
df.head()

,name,price,link,specs_combined
0,ثلاجة بفريزر علوي وايت بوينت، نوفروست، 420 لتر...,"27,599",https://btech.com/ar/white-point-nofrost-refri...,البراند: وايت بوينت\nالنوع: ثلاجة بفريزر علوي\...
1,ثلاجة بفريزر علوي تورنيدو، نوفروست، 450 لتر، ا...,"23,799",https://btech.com/ar/tornado-no-frost-top-free...,البراند: تورنيدو\nالنوع: ثلاجة بفريزر علوي\nنو...
2,ثلاجة بفريزر علوي تورنيدو، نوفروست، 396 لتر، ا...,"21,999",https://btech.com/ar/tornado-no-frost-refriger...,البراند: تورنيدو\nالنوع: ثلاجة بفريزر علوي\nنو...
3,ثلاجة بفريزر علوي تورنيدو، نوفروست، 396 لتر، ا...,"21,999",https://btech.com/ar/tornado-nofrost-refrigera...,البراند: تورنيدو\nالنوع: ثلاجة بفريزر علوي\nنو...
4,ثلاجة بفريزر علوي تورنيدو، نوفروست، 396 لتر، س...,"23,499",https://btech.com/ar/tornado-nofrost-refrigera...,البراند: تورنيدو\nالنوع: ثلاجة بفريزر علوي\nنو...


# **Step 2: Data Cleaning & Preprocessing**

## a. Dependancies

In [ ]:
!pip install stanza pyarabic --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59

In [ ]:
!pip install stanza arabic-stopwords --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 kB 6.1 MB/s eta 0:00:00


In [ ]:
# Imports
import string
import re
import stanza
import arabicstopwords.arabicstopwords as stp

## b. stanza and stop-words intilization

In [ ]:
stanza.download('ar')
nlp = stanza.Pipeline(lang="ar", processors="tokenize,lemma", verbose=False, use_gpu=False)
arabic_stopwords = set(stp.stopwords_list())
arabic_punct     = "،؛؟ـ«»…"
all_punct        = string.punctuation + arabic_punct
translator       = str.maketrans('', '', all_punct)

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: ar (Arabic) ...


INFO:stanza:Downloaded file to /root/stanza_resources/ar/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


## c. main cleaning functions

In [ ]:
def tokens_to_text(text_or_tokens):
    return ' '.join(text_or_tokens) if isinstance(text_or_tokens, list) else text_or_tokens

def remove_tashkeel(text):
    tashkeel_pattern = r'[\u0610-\u061A\u064B-\u0652\u0670\u0640]'
    return re.sub(tashkeel_pattern, '', text)

def remove_stopwords(text):
    return " ".join(w for w in text.split() if w not in arabic_stopwords)

def remove_punctuation(text):
    return text.translate(translator)

def lemmatize_text(text):
    if not isinstance(text, str):
        return text
    doc = nlp(text)
    return " ".join(w.lemma for s in doc.sentences for w in s.words)

def clean_text(text_or_tokens):
    text = tokens_to_text(text_or_tokens)
    text = remove_stopwords(text)
    text = remove_punctuation(text)
    text = lemmatize_text(text)
    text = remove_tashkeel(text)
    text = remove_stopwords(text)
    return text

In [ ]:
df['name_clean'] = df['name'].apply(clean_text)
df['specs_text_clean'] = df['specs_combined'].apply(clean_text)
df.head()

,name,price,link,specs_combined,name_clean,specs_text_clean
0,شاشة تلفزيون سامسونج سمارت 55 بوصة بدقة 4K UHD...,"19,099",https://btech.com/ar/samsung-55-4k-uhd-smart-l...,البراند: سامسونج\nحجم الشاشة: 55 بوصة\nنوع الت...,شاشة تلفزيون سامسونج سمارت 55 بوصة بدقة 4K UHD...,البراند سامسونج حجم شاشة 55 بوصة نوع تلفزيون س...
1,شاشة تلفزيون سامسونج سمارت 43 بوصة LED بدقة FH...,"13,500",https://btech.com/ar/samsung-43-fhd-smart-led-...,البراند: سامسونج\nنوع الدقة: FHD\nنوع الشاشة: ...,شاشة تلفزيون سامسونج سمارت 43 بوصة LED بدقة FH...,البراند سامسونج نوع دقة FHD نوع شاشة LED اتصال...
2,شاشة سامسونج 32 بوصة سمارت LED ، دقة HD، بريسي...,"8,323",https://btech.com/ar/samsung-32-hd-smart-led-u...,البراند: سامسونج\nنوع الدقة: HD (1366×768)\nال...,شاشة سامسونج 32 بوصة سمارت LED دقة HD بريسيف د...,البراند سامسونج نوع دقة HD 1366 ×768 اتصال 1 م...
3,تلفزيون سمارت ال جي 55 بوصة، LED، دقة 4K UHD، ...,"22,799",https://btech.com/ar/lg-tv-55-inch-4k-smart-le...,البراند: ال جي\nحجم الشاشة: 55 بوصة\nنوع التلف...,تلفزيون سمارت ال جي 55 بوصة LED دقة 4K UHD بري...,البراند ال جي حجم شاشة 55 بوصة نوع تلفزيون تلف...
4,تلفزيون ال جي، 70 بوصة، سمارت LED، دقة 4K UHD،...,"31,999",https://btech.com/ar/lg-70-inch-4k-smart-led-t...,البراند: ال جي\nنوع التلفزيون: سمارت\nنوع الشا...,تلفزيون ال جي 70 بوصة سمارت LED دقة 4K UHD بري...,البراند ال جي نوع تلفزيون سمارت نوع شاشة LED ا...


In [ ]:
# df.to_csv("btech_mobile_phones_final_cleaned.csv", index=False, encoding='utf-8-sig')

# **Step 3: Arabic Embeddings Generation using AraBERT**

## a. dependancies

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")
model = AutoModel.from_pretrained("asafaya/bert-base-arabic")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(32000, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

## b. embedding Generation Function

In [ ]:
def generate_arabert_embeddings(texts, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=512
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)

        token_embeddings = outputs.last_hidden_state
        input_mask_expanded = inputs['attention_mask'].unsqueeze(-1).expand(token_embeddings.size()).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
        sum_mask = torch.clamp(input_mask_expanded.sum(dim=1), min=1e-9)
        batch_embeddings = sum_embeddings / sum_mask
        embeddings.extend(batch_embeddings.cpu().numpy())
    return np.array(embeddings)

## c. embeddings for cleaned/processed columns

In [ ]:
df['name_embeddings'] = pd.Series(generate_arabert_embeddings(df['name_clean'].tolist()).tolist())
df['specs_embeddings'] = pd.Series(generate_arabert_embeddings(df['specs_text_clean'].tolist()).tolist())
df.head()

In [ ]:
df.to_csv("BTech_TVs_embedded.csv", index=False, encoding='utf-8-sig')

In [ ]:
# Filter rows where both embeddings exist and are valid
valid_rows = df[
    df[["specs_embeddings", "name_embeddings"]].notnull().all(axis=1)
]

# Optional: Log invalid rows for debugging
invalid_rows = df[
    df[["specs_embeddings", "name_embeddings"]].isnull().any(axis=1)
]
print("Invalid rows (missing embeddings):", len(invalid_rows))

Invalid rows (missing embeddings): 0


In [ ]:
# Check for empty or invalid embeddings
for col in ["specs_embeddings", "name_embeddings"]:
    if df[col].isnull().any():
        print(f"⚠️ Null values found in `{col}`")
    if df[col].apply(lambda x: len(x) != 768 if isinstance(x, list) else True).any():
        print(f"⚠️ Invalid dimension in `{col}` (not 768)")

### **NB (skip the below cells)**: If CSV data needs to be uploaded, it should first be converted to a binary format or a Parquet file.

In [ ]:
#from google.colab import files
#uploaded = files.upload()


In [ ]:
#df = pd.read_csv('elaraby_products_with_specs_final_cleaned_embedded.csv')
#df.head()

In [ ]:
#type(df['specs_embeddings'].iloc[1])

# **Step 4: Vector Database + Metadata Storage**

## a.dependancies

In [ ]:
!pip install qdrant-client --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.0/329.0 kB 6.5 MB/s eta 0:00:00


In [ ]:
!pip install -U qdrant-client --q

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance
from qdrant_client.http.models import Batch
import uuid

## b. VDB creation

In [ ]:
qdrant_client = QdrantClient(
    url="https://9052b275-2c38-486f-b97b-a4fa134b4141.us-east-1-0.aws.cloud.qdrant.io:6333",
    api_key="--",)

In [ ]:
qdrant_client.create_collection(
    collection_name="B-Tech",
    vectors_config={
        "specs": VectorParams(size=768, distance=Distance.COSINE),
        "name": VectorParams(size=768, distance=Distance.COSINE)
    }
)

True

In [ ]:
print(qdrant_client.get_collections())

collections=[CollectionDescription(name='B-Tech'), CollectionDescription(name='elaraby-products')]


## c. Inserting embeddings into the VDB

### i. preparing points for upsert

In [ ]:
points = []
for idx, row in df.iterrows():
    points.append({
        "id": str(uuid.uuid4()),
        "vector": {
            "specs": row["specs_embeddings"],
            "name": row["name_embeddings"]
        },
        "payload": {
            "name": row["name"],
            "price": row["price"],
            "link": row["link"],
            "category": "TVs"
        }
    })

In [ ]:
print(type(df["specs_embeddings"].iloc[0]))
print(len(df["specs_embeddings"].iloc[0]))

<class 'list'>
768


In [ ]:
points[0]

{'id': 0,
 'vector': {'specs': [[0.181781604886055,
    -0.6339461803436279,
    0.044391825795173645,
    -0.4803466796875,
    0.1474595069885254,
    0.013694604858756065,
    -0.12909537553787231,
    -0.09444650262594223,
    -0.060410354286432266,
    0.1138446033000946,
    -0.15196876227855682,
    0.43436896800994873,
    0.4538816213607788,
    0.0677887573838234,
    0.4457477927207947,
    -0.2120857983827591,
    -0.07164092361927032,
    -0.014877998270094395,
    -0.07855992019176483,
    -0.4430187940597534,
    -0.5089349150657654,
    -0.22927328944206238,
    0.8001728653907776,
    0.11870194971561432,
    -0.37258923053741455,
    0.3051833510398865,
    -0.025939492508769035,
    -0.05687740072607994,
    0.19693392515182495,
    0.07718226313591003,
    0.02038712240755558,
    -0.22326745092868805,
    -0.22144168615341187,
    -0.6705882549285889,
    -0.6068269610404968,
    -0.5831062197685242,
    -0.24876968562602997,
    -0.2660788297653198,
    -0.3103483

In [ ]:
len(points)

120

In [ ]:
collection_info = qdrant_client.get_collection("B-Tech")
print(collection_info.config)

params=CollectionParams(vectors={'name': VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), 'specs': VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None)}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None) hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None) optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None) wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0) quantization_config=None strict_mode_conf

### ii. Upserting into the created collection

In [ ]:
qdrant_client.upsert(collection_name="B-Tech", points=points)

UpdateResult(operation_id=6, status=<UpdateStatus.COMPLETED: 'completed'>)

### iii. check the results

In [ ]:
resp = qdrant_client.count(collection_name="elaraby-products")
print(f"Total points in collection: {resp.count}")

Total points in collection: 12


In [ ]:
sample = qdrant_client.retrieve(
    collection_name="elaraby-products",
    ids=[0],
    with_payload=True,
    with_vectors=True
)

pt = sample[0]
print("ID:", pt.id)
print("Payload:", pt.payload)
print("Vector length:", len(pt.vector))

ID: 0
Payload: {'name': 'ثلاجة تورنيدو ديفروست 100 لتر ميني بار سيلفر MBR-AR100-S', 'price': 'ج.م 10,945', 'link': 'https://www.elarabygroup.com/ar/tornado-refrigerator-defrost-100-liter-1-door-mini-bar-in-silver-color-mbr-ar100-s'}
Vector length: 768


# **Step 5: Query Vectorization & Top-k Product Retrieval**

In [ ]:
def show_nearest_products(df, qdrant_client, idx: int, k: int = 5):
    query_vec     = df.loc[idx, "specs_embeddings"].tolist()
    expected_name = df.at[idx, "name"]
    print("Expected product name:", expected_name)
    results = qdrant_client.search(
        collection_name="elaraby-products",
        query_vector=query_vec,
        limit=k,
        with_payload=True
    )
    print(f"\nTop {k} nearest to item {idx}:")
    for hit in results:
        print(f" • ID={hit.id}, score={hit.score:.3f}, name={hit.payload['name']}")

In [ ]:
show_nearest_products(df, qdrant_client, idx=5, k=7)

# **Step 6: LLM + RAG Prompt Builder**

## a.dependancies

In [ ]:
import openai
from openai import OpenAI
from google.colab import files
import pandas as pd

In [ ]:
!pip install qdrant-client --upgrade --q

In [ ]:
from openai import OpenAI
openai_client = OpenAI(api_key="--")

In [ ]:
openai.api_key = "OpenAI_token"

## b. RAG + LLM

In [ ]:
#  RAG PROMPT BUILDER

def build_rag_prompt(user_query, qdrant_client, top_k=5, vector_field="specs"):
    """Builds a RAG prompt using Qdrant vector search results"""
    # Clean and embed query
    cleaned = clean_text(user_query)
    q_emb = generate_arabert_embeddings([cleaned])[0].tolist()

    # Use updated Qdrant API with query_points
    results = qdrant_client.query_points(
        collection_name="B-Tech",
        using=vector_field,  # First parameter
        query=q_emb,                # Second parameter
        limit=top_k,
        with_payload=True
    )

    # Extract relevant products with validation
    retrieved_products = []
    for hit in results.points:
        retrieved_products.append({
            "name": hit.payload["name"],
            "price": hit.payload["price"],
            "link": hit.payload["link"]
        })

    # Build structured prompt
    prompt = f"User Query: {user_query}\n\nRelevant Products:\n"
    for i, p in enumerate(retrieved_products, 1):
        prompt += f"{i}. Name: {p['name']}\n   Price: {p['price']}\n   Link: {p['link']}\n\n"

    prompt += "Please provide a detailed response in Arabic based on the above information."
    return prompt, retrieved_products

In [ ]:
p,r = build_rag_prompt("أريد معلومات عن ثلاجات", qdrant_client, top_k=5, vector_field="specs")
print(r)

[{'name': 'لوح رسم LCD ديجيتال، 8.5 بوصة، قلم للمسح الذاتي - اسود', 'price': '99', 'link': 'https://btech.com/ar/digital-drawing-board-lcd-8-5-inch-self-erase-pen.html'}, {'name': 'غسالة اطباق بيكو، 13 فرد، ستانلس ستيل - DVN05325X', 'price': '21,739', 'link': 'https://btech.com/ar/beko-freestanding-dishwasher-13-persons-stainless-steel-dvn05325x.html'}, {'name': 'غسالة اطباق تورنيدو، 13 فرد، اسود - TDV-FN138CBK', 'price': '25,579', 'link': 'https://btech.com/ar/tornado-dishwasher-13-persons-black-tdv-fn138cbk.html'}, {'name': 'بوتجاز غاز فريش بونتو، 5 شعلات، فضي واسود - 17304', 'price': '11,960', 'link': 'https://btech.com/ar/fresh-punto-gas-cooker-black-silver-5burners-17304.html'}, {'name': 'غسالة اطباق تورنيدو، 12 فرد، اسود - TDV-FN128CBK', 'price': '26,699', 'link': 'https://btech.com/ar/tornado-dishwasher-12-persons-black-tdv-fn128cbk.html'}]


In [ ]:
# SYSTEM PROMPT & FEW-SHOT EXAMPLES

SYSTEM_PROMPT = (
    "أنت مساعد ذكي ومتخصص في توصيف منتجات الأجهزة الكهربائية. "
    "عندما يستقبل استفساراً عن نوع جهاز، فعليك:\n"
    "  1. التأكد من أن المنتجات المدرجة تتناسب مع نوع الاستعلام (مثال: إذا سُئلت عن غسّالات، فلا تدرج ديب فريزر).\n"
    "  2. سرد أفضل ثلاث إلى خمس منتجات بترتيب الصلاحية (الأكثر ملاءمة أولاً).\n"
    "  3. لكل منتج، أعطِ الاسم، السعة أو الميزة الأهم، السعر، ورابط الشراء.\n"
    "  4. أختم بردٍ موجز يشرح لماذا هي الأنسب."
)

FEW_SHOT_EXAMPLES = [
    {"role": "user", "content": "أريد معلومات عن ثلاجات."},
    {"role": "assistant", "content":
        "إليك أفضل 3 ثلاجات تناسب احتياجاتك:\n\n"
        "1. **ثلاجة سامسونج 18 قدم** — سعة 510 لتر، فئة الطاقة A+ — 12,999 ج.م — https://…\n"
        "2. **ثلاجة إل جي 16 قدم** — سعة 450 لتر، فئة الطاقة A — 10,750 ج.م — https://…\n"
        "3. **ثلاجة كريازي 14 قدم** — سعة 390 لتر، فئة الطاقة A+ — 9,499 ج.م — https://…\n\n"
        "اخترت هذه الثلاجات لأنها توفر سعات مختلفة مع كفاءة طاقة عالية."
    },
    {"role": "user", "content": "أحتاج توصية عن غسّالة."},
    {"role": "assistant", "content":
        "هذه أفضل 3 غسّالات حسب سعة الاستخدام والأداء:\n\n"
        "1. **غسّالة سامسونج أمامية 9 كغم** — سرعة دوران 1400 لفة/دقيقة — 8,499 ج.م — https://…\n"
        "2. **غسّالة إل جي أمامية 8 كغم** — تقنية البخار Steam+ — 7,299 ج.م — https://…\n"
        "3. **غسّالة وايت بوينت تحميل علوي 10 كغم** — توفير كهرباء — 6,999 ج.م — https://…\n\n"
        "اخترت هذه الغسّالات لما تقدمه من سعات وتقنيات ملائمة للغسيل اليومي."
    },
]

In [ ]:
#  LLM CALL WITH FEW-SHOT

def get_llm_response_with_fewshot(prompt_text):
    """Get response from GPT with few-shot prompting"""
    try:
        messages = [
            {"role": "system", "content": SYSTEM_PROMPT},
            *FEW_SHOT_EXAMPLES,
            {"role": "user", "content": prompt_text}
        ]

        response = openai_client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            max_tokens=500,
            temperature=0.3,
            top_p=0.9
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"عذرًا، حدث خطأ في معالجة الطلب: {str(e)}"

In [ ]:
#  END-TO-END RESPONSE GENERATOR

def generate_response(user_query, qdrant_client, top_k=5, vector_field="specs"):
    """Generate complete response from user query"""
    prompt, retrieved_products = build_rag_prompt(
        user_query, qdrant_client, top_k, vector_field
    )

    if not retrieved_products:
        return "عذرًا، لم نتمكن من العثور على منتجات تطابق طلبك.", []

    return get_llm_response_with_fewshot(prompt), retrieved_products

## c. Test

In [ ]:
reply, products = generate_response("عاوز ثلاجة توشيبا رخيص ", qdrant_client, top_k=5)
print("Generated Reply:\n", reply)

Generated Reply:
 عذرًا، لم أعثر على ثلاجة توشيبا بسعر منخفض في القائمة المتاحة. ومع ذلك، إذا كنت تبحث عن ثلاجة بسعر معقول، يمكنك النظر في الخيارات الأخرى المدرجة أعلاه. ثلاجة نوفروست يونيون اير سيجنتشر بسعر 17,569 ج.م أو ثلاجة نوفروست يونيون اير سيجنتشر بسعر 24,900 ج.م قد تكونا خيارات جيدة لك.


In [ ]:
reply, products = generate_response("عاوز تليفيزيون رخيص ", qdrant_client, top_k=5)
print("Generated Reply:\n", reply)

Generated Reply:
 بناءً على استعلامك عن تلفزيون رخيص، إليك بعض الخيارات المناسبة:

1. **لوح رسم LCD ديجيتال، 8.5 بوصة، قلم للمسح الذاتي - أسود**
   السعر: 99 ج.م
   رابط الشراء: [اضغط هنا](https://btech.com/ar/digital-drawing-board-lcd-8-5-inch-self-erase-pen.html)

2. **شاشة تلفزيون ايفولف، 32 بوصة، LED، بدقة HD، بدون إطار**
   السعر: 3,999 ج.م
   رابط الشراء: [اضغط هنا](https://btech.com/ar/evolve-32-inch-fhd-standard-led-frameless-tv.html)

3. **غسالة اطباق بوش سيريز 4، سعة 13 فرد، رمادي غامق - SMS4IKC62T**
   السعر: 43,200 ج.م
   رابط الشراء: [اضغط هنا](https://btech.com/ar/bosch-series4-freestanding-dishwasher-13-persons-sms4ikc62t.html)

تم اختيار هذه الخيارات بناءً على توفرها وأسعارها المناسبة لميزانيتك.


In [ ]:
!pip freeze

absl-py==1.4.0
accelerate==1.8.1
aiofiles==24.1.0
aiohappyeyeballs==2.6.1
aiohttp==3.11.15
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.24
albumentations==2.0.8
ale-py==0.11.1
altair==5.5.0
annotated-types==0.7.0
antlr4-python3-runtime==4.9.3
anyio==4.9.0
argon2-cffi==25.1.0
argon2-cffi-bindings==21.2.0
array_record==0.7.2
arviz==0.21.0
astropy==7.1.0
astropy-iers-data==0.2025.6.23.0.39.50
astunparse==1.6.3
atpublic==5.1
attrs==25.3.0
audioread==3.0.1
autograd==1.8.0
babel==2.17.0
backcall==0.2.0
backports.tarfile==1.2.0
beautifulsoup4==4.13.4
betterproto==2.0.0b6
bigframes==2.7.0
bigquery-magics==0.9.0
bleach==6.2.0
blinker==1.9.0
blis==1.3.0
blobfile==3.0.0
blosc2==3.4.0
bokeh==3.7.3
Bottleneck==1.4.2
bqplot==0.12.45
branca==0.8.1
build==1.2.2.post1
CacheControl==0.14.3
cachetools==5.5.2
catalogue==2.0.10
certifi==2025.6.15
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.2
chex==0.1.89
clarabel==0.11.1
click==8.2.1
cloudpathlib==0.21.1
cloudpickle==3.1.1
cmake==3.31.6
cmdstanp

In [ ]:
reply, products = generate_response("اقترح افضل تابلت عندك", qdrant_client, top_k=5)
print("Generated Reply:\n", reply)

Generated Reply:
 بناءً على طلبك، إليك أفضل تابلت من بين المنتجات المعروضة:

**لوح رسم LCD ديجيتال، 8.5 بوصة، قلم للمسح الذاتي - اسود**
- السعر: 99 ج.م
- الرابط: [اشتر الآن](https://btech.com/ar/digital-drawing-board-lcd-8-5-inch-self-erase-pen.html)

تم اختيار هذا اللوح الرقمي بسبب سعره المناسب وميزة القلم للمسح الذاتي التي تجعله مثاليًا للاستخدام اليومي والرسم الإبداعي.


# FAQs

In [ ]:
df = pd.read_csv("FAQs.csv")
df.head()

,Question,Answer
0,س1: ماذا تعني سياسة الاستبدال والاسترجاع في بي...,ج: سياسة الاستبدال والاسترجاع تنظم شروط استبدا...
1,س2: هل يمكنني تعديل أو إلغاء طلبي بعد الشراء؟,ج: نعم، يمكنك تعديل أو إلغاء طلب الشراء خلال 7...
2,س3: ما هي الشروط لاستبدال أو استرجاع المنتجات؟,ج: تشمل الشروط الرئيسية:\nتقديم الطلب خلال 14 ...
3,س4: ماذا أفعل إذا استلمت منتج معيب أو غيرت رأيي؟,ج: يتم تقديم طلب الاسترجاع من خلال خدمة العملا...
4,س5: ما هي شروط استبدال أو استرجاع المنتجات الم...,ج: الشروط تتضمن:\nطلب الاستبدال أو الاسترجاع خ...


In [ ]:
df['Question_clean'] = df['Question'].apply(clean_text)
df.head()

,Question,Answer,Question_clean
0,س1: ماذا تعني سياسة الاستبدال والاسترجاع في بي...,ج: سياسة الاستبدال والاسترجاع تنظم شروط استبدا...,س1 عنى سياسة الاستبدال استرجاع تك
1,س2: هل يمكنني تعديل أو إلغاء طلبي بعد الشراء؟,ج: نعم، يمكنك تعديل أو إلغاء طلب الشراء خلال 7...,س2 يمكنني تعديل إلغاء طلبي شراء
2,س3: ما هي الشروط لاستبدال أو استرجاع المنتجات؟,ج: تشمل الشروط الرئيسية:\nتقديم الطلب خلال 14 ...,س3 شرط استبدال استرجاع منتج
3,س4: ماذا أفعل إذا استلمت منتج معيب أو غيرت رأيي؟,ج: يتم تقديم طلب الاسترجاع من خلال خدمة العملا...,س4 أفعل استلمة منتج معيب غيرت رأيي
4,س5: ما هي شروط استبدال أو استرجاع المنتجات الم...,ج: الشروط تتضمن:\nطلب الاستبدال أو الاسترجاع خ...,س5 شرط استبدال استرجاع منتج المعيبة


In [ ]:
df['Question_embeddings'] = pd.Series(generate_arabert_embeddings(df['Question_clean'].tolist()).tolist())
df.head()

,Question,Answer,Question_clean,Question_embeddings
0,س1: ماذا تعني سياسة الاستبدال والاسترجاع في بي...,ج: سياسة الاستبدال والاسترجاع تنظم شروط استبدا...,س1 عنى سياسة الاستبدال استرجاع تك,"[0.3422534167766571, -1.0445961952209473, 0.13..."
1,س2: هل يمكنني تعديل أو إلغاء طلبي بعد الشراء؟,ج: نعم، يمكنك تعديل أو إلغاء طلب الشراء خلال 7...,س2 يمكنني تعديل إلغاء طلبي شراء,"[0.4110267758369446, -0.21164026856422424, -0...."
2,س3: ما هي الشروط لاستبدال أو استرجاع المنتجات؟,ج: تشمل الشروط الرئيسية:\nتقديم الطلب خلال 14 ...,س3 شرط استبدال استرجاع منتج,"[0.35406357049942017, -0.5212626457214355, 0.0..."
3,س4: ماذا أفعل إذا استلمت منتج معيب أو غيرت رأيي؟,ج: يتم تقديم طلب الاسترجاع من خلال خدمة العملا...,س4 أفعل استلمة منتج معيب غيرت رأيي,"[0.5459046363830566, -0.6037685871124268, -0.2..."
4,س5: ما هي شروط استبدال أو استرجاع المنتجات الم...,ج: الشروط تتضمن:\nطلب الاستبدال أو الاسترجاع خ...,س5 شرط استبدال استرجاع منتج المعيبة,"[0.2329837828874588, -0.6482943296432495, 0.14..."


In [ ]:
qdrant_client.create_collection(
    collection_name="FAQs",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE)
)

True

In [ ]:
points = []
for idx, row in df.iterrows():
    points.append({
        "id": str(uuid.uuid4()),
        "vector": row["Question_embeddings"],
        "payload": {
            "question": row["Question"],
            "answer": row["Answer"]
        }
    })

In [ ]:
qdrant_client.upsert(collection_name="FAQs", points=points)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [ ]:
# Build RAG prompt for FAQs
def build_rag_prompt_for_faqs(user_query, qdrant_client, top_k=5):
    cleaned_query = clean_text(user_query)
    query_embedding = generate_arabert_embeddings([cleaned_query])[0].tolist()

    results = qdrant_client.query_points(
        collection_name="FAQs",
        query=query_embedding,
        limit=top_k,
        with_payload=True
    )

    retrieved_faqs = []
    for hit in results.points:
        retrieved_faqs.append({
            "question": hit.payload["question"],
            "answer": hit.payload["answer"]
        })

    prompt = f"User Query: {user_query}\n\nRelevant FAQs:\n"
    for idx, faq in enumerate(retrieved_faqs, start=1):
        prompt += f"{idx}. Question: {faq['question']}\n   Answer: {faq['answer']}\n\n"

    prompt += "Please provide a detailed response in Arabic based on the above information."
    return prompt, retrieved_faqs

# Generate FAQ response
def generate_faq_response(user_query, qdrant_client, top_k=5):
    prompt, _ = build_rag_prompt_for_faqs(user_query, qdrant_client, top_k)
    llm_response = get_llm_response_with_fewshot(prompt)
    return llm_response

In [ ]:
# Test the system
user_query = "كيفية استرجاع المنتجات؟"
reply = generate_faq_response(user_query, qdrant_client)
print("Generated Reply:\n", reply)

Generated Reply:
 يمكنك استرجاع المنتجات وفقًا للشروط التالية:
- يجب تقديم الطلب خلال 14 يومًا من تاريخ الاستلام للمنتجات العادية.
- يجب أن يكون المنتج في حالته الأصلية وغير مستخدم، مع العبوة الأصلية والفاتورة.
- يجب أن يكون الرقم التسلسلي واضحًا وغير متلاعب به، ويجب أن تكون جميع الملحقات مرفقة.

يمكن للعملاء إرجاع المنتجات في غضون 14 إلى 30 يومًا من تاريخ التسليم لاسترداد كامل المبلغ. بعد إصدار تقرير فني يؤكد العيب، يمكنك استبدال أو استرجاع المنتج خلال 7 أيام من تاريخ الإخطار، مع تغطية كافة مصاريف الشحن.

سيتم رد ثمن المنتج بنفس طريقة الدفع المسجلة في الفاتورة، أو من خلال رصيد في حسابك على الموقع، أو بقسائم شراء، أو تحويل بنكي، أو أي طريقة أخرى تراها بي تك مناسبة.

قبل إرجاع الأجهزة الإلكترونية، يجب إزالة جميع المعلومات المخزنة عليها، وإل


In [ ]:
user_query = "امتي افقد حق ارجاع"
reply = generate_faq_response(user_query, qdrant_client)
print("Generated Reply:\n", reply)

Generated Reply:
 إذا كنت ترغب في إرجاع منتج معيب، يجب أن تلتزم بالشروط التالية:
- يجب طلب الاستبدال أو الاسترجاع خلال 30 يوماً من تاريخ الاستلام.
- يجب أن يكون المنتج في حالته الأصلية ويجب أن تكون الفاتورة مرفقة.
- لا يجب أن يكون هناك نقص في الملحقات.
- يجب عدم تلف المنتج بسبب سوء الاستخدام أو التركيب.

إذا كان المنتج تحت وكالة بي تك، سيقوم فريق الدعم الفني بفحص الجهاز وبعد التأكد من مشكلة العيب، سيتم كتابة تقرير فني بالعيب، وبعدها يمكنك استرداد المبلغ أو تغيير المنتج بآخر إن وجد.

إذا كان المنتج تابعًا لوكيل آخر، يتم الاتصال بالوكيل لتحديد موعد لفحص المنتج وعمل تقرير فني بالعيب، وبعد التأكد من مشكلة العيب، سيتم كتابة تقرير فني بالعيب، وبعدها يمكنك استرداد المبلغ أو تغيير المنتج بآخر إن وجد.

يرجى الالتزام بالشروط المذكورة لضمان إرجاع المنت


In [ ]:
user_query = "يعني ايه حالة الفرز الأول"
reply = generate_faq_response(user_query, qdrant_client)
print("Generated Reply:\n", reply)

Generated Reply:
 حالة الفرز الأول تعني أن المنتج هو منتج جديد ولم يتم استخدامه من قبل، كما أنه لم يتم فتحه ولا يوجد به أي تلف. يجب أن يكون المنتج في عبوته الأصلية التي تم تسليمها بها، ويجب على العميل الاحتفاظ بها كما هي. يمكن للعملاء إرجاع المنتجات التي تكون في حالة الفرز الأول خلال فترة تتراوح بين 14 إلى 30 يومًا من تاريخ التسليم لاسترداد كامل المبلغ.


In [ ]:
!pip install openai qdrant-client pandas
!pip install -q sentence-transformers qdrant-client


In [ ]:
openai.api_key = "--"

In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding


In [ ]:
QDRANT_URL = "https://9052b275-2c38-486f-b97b-a4fa134b4141.us-east-1-0.aws.cloud.qdrant.io:6333"
QDRANT_API_KEY = "--"
COLLECTION = "new_BTECH_collection"

client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)


In [ ]:
def query_embedding(query):
    response = openai.embeddings.create(
        input=query,
        model="text-embedding-3-small"
    )
    return response.data[0].embedding

def search_products(query, k=5):
    q_vec = query_embedding(query)
    hits = client.search(collection_name=COLLECTION, query_vector=q_vec, limit=k)
    for h in hits:
        print(f"Score: {h.score:.3f}")
        print(f"{h.payload['name']} — {h.payload['price']}")
        print(h.payload['specs'])
        print(h.payload['link'], "\n")

search_products(" قارن بين موبايل شاومي نوت 14 و ريلمي C53، سعة 128 جيجا", k=10)


Score: 0.744
شاومي نوت 14، سعة 128 جيجا، رام 8 جيجا، شبكة 4G، بشريحتين - اخضر نعناعي، اصدار محلي — 10,170
البراند: شاومي
الموديل: نوت 14
الشاشة: - شاشة أموليد، مقاس 6.67 بوصة، بمعدل تحديث 120 هرتز | - الدقة: 2400×1080
الكاميرا الخلفية: - كاميرا ثلاثية | - دقة 108 ميجا بكسل، فتحة عدسة f/1.7، 1.1/67 بوصة، 0.64 مايكرو متر، تدعم تقنية تجميع البكسلات 9 في 1 1.92 مايكرو متر | - دقة 2 ميجا بكسل، فتحة عدسة f/2.4، (عدسة عُمق) | - دقة 2 ميجا بكسل، فتحة عدسة f/2.4، (عدسة ماكرو) | - دقة تصوير الفيديو: 1080 بكسل بمعدل 30/60 إطار في الثانية، و 720 بكسل بمعدل 30 إطار في الثانية
الكاميرا الأمامية: - دقة 20 ميجا بكسل، فتحة عدسة f/2.2، 1/4 بوصة، مكونة من 4 عناصر | - دقة تصوير الفيديو: 1080 بكسل بمعدل 30 إطار في الثانية، و720 بكسل بمعدل 30 إطار في الثانية
سعة الذاكرة: 128 جيجا بايت - الرام 8 جيجا بايت
المعالج: هيليو G99 - الترا 6 نانو متر - ثماني النواة
البطارية: - سعة البطارية 5500 مللي أمبير | - تدعم خاصية الشحن السريع بقدرة 33 وات
عدد الشرائح: شريحتان
الاتصال: واي فاي، بلوتوث 5.3، منفذ USB-C، GPS، جلو

/tmp/ipython-input-39-3560404392.py:10: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  hits = client.search(collection_name=COLLECTION, query_vector=q_vec, limit=k)
